In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# TensorFlow: Convert TFRecords to Parquet files

## TFRecords

[TFRecords](https://www.tensorflow.org/tutorials/load_data/tfrecord) are a popular file format to store data for deep learning training with TensorFlow. It is a "simple format for storing a sequence of binary records". In many cases the dataset is too large for the host memory and the dataset is converted into (multiple) tfrecords file to disk. TensorFlow's ecosystem enables to stream the tfrecords from disk to train the model without requiring to load the full dataset.<br><br>
That sounds great, but there are some disadvantages when working with tabular dataset. TFRecords stores the dataset as key, values. In other domains, such as computer vision, this representation is efficient as the key is `image` and the values are a the pixels. For an RGB image with 200x200 resoultion, there are 120000 (200x200x3) values. In a tabular dataset, a feature is often a single number and therefore, there is a significant overhead for using a key in each example. **In some of our experiments, we experienced that tfrecords can be ~4-5x larger than `parquet` files for the same dataset.**
<br><br>
[Parquet](https://en.wikipedia.org/wiki/Apache_Parquet) is another file format to store data. It is a free and open-source data storage format in the Hadoop ecosystem. Many popular systems, such as Spark or Pandas, support to read and write parquet files. 
<br><br>
We developed [NVTabular Data Loaders](https://nvidia.github.io/NVTabular/main/training/index.html) as a customized data loader, fully operating on the GPU. It reads the data from disk into the GPU memory and prepares the next batch on the GPU. Therefore, we do not have any CPU-GPU communication. Our data loader leverages parquet files to reduce the disk pressure. **In our experiments, we experienced that the native data loader is the bottleneck in training tabular deep learning models and by changing the native data loader to NVTabular Data Loader, we saw a 8-9x speed-up.**

### Convert TFRecords to Parquet files
That is a lot of background information. In many cases, we saw that users have their dataset stored as tfrecords files. In this notebook, we provide a tfrecords to parquet examples. Users can transform their dataset to parquet and be able to experiment with NVTabular data loader.

We leverage the library pandas-tfrecords.

In [2]:
!pip install pandas-tfrecords==0.1.5

distutils: /usr/local/lib/python3.8/dist-packages
sysconfig: /usr/lib/python3.8/site-packages
distutils: /usr/local/lib/python3.8/dist-packages
sysconfig: /usr/lib/python3.8/site-packages
distutils: /usr/local/include/python3.8/UNKNOWN
sysconfig: /usr/include/python3.8/UNKNOWN
distutils: /usr/local/bin
sysconfig: /usr/bin
distutils: /usr/local
sysconfig: /usr
user = False
home = None
root = None
prefix = None
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |██████████████████████████▍     | 374.7 MB 94.2 MB/s eta 0:00:01

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |███████████████████████████████▍| 445.9 MB 85.6 MB/s eta 0:00:01

     |███████████████████████████████▊| 450.6 MB 85.6 MB/s eta 0:00:01

     |████████████████████████████████| 454.4 MB 85.6 MB/s 


     |████████████████████████████████| 462 kB 66.9 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 54.3 MB/s eta 0:00:01


  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.39.0
    Uninstalling grpcio-1.39.0:
      Successfully uninstalled grpcio-1.39.0
  distutils: /usr/local/include/python3.8/grpcio
  sysconfig: /usr/include/python3.8/grpcio
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  distutils: /usr/local/include/python3.8/tensorflow-estimator
  sysconfig: /usr/include/python3.8/tensorflow-estimator
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.0
    Uninstalling tensorflow-2.6.0:
      Successfully uninstalled tensorflow-2.6.0
  distutils: /usr/local/include/python3.8/tensorflow
  sysconfig: /usr/include/python3.8/tensorflow
  distutils: /usr/local/include/python3.8/pandas-tfrecords
  sysconfig: /usr/include/python3.8/pandas-tfrecords
ERROR: pip's dependency resolver does not cur

## Create a Synthetic Dataset

First, we will create a synthetic dataset. Afterwards, we will convert the synthetic data to a tfrecord file. The synthetic dataset contains `continuous features`, `categorical features`, `continuous features in a list with variable length`, `categorical features in a list with variable length` and the `label`.<br><br>
The features of a list have variable length, which are often used in session-based recommender systems. For example, the last page views in a session and sessions have different lengths.

In [3]:
import numpy as np
import pandas as pd

In [4]:
def create_synthetic_df(
    N_CONT_FEATURES, N_CAT_FEATURES, N_CONT_LIST_FEATURES, N_CAT_LIST_FEATURES, N_ROWS
):
    dict_features = {}
    for icont in range(N_CONT_FEATURES):
        dict_features["cont" + str(icont)] = np.random.uniform(-1, 1, size=N_ROWS)
    for icat in range(N_CAT_FEATURES):
        dict_features["cat" + str(icat)] = np.random.choice(list(range(10)), size=N_ROWS)
    for icontlist in range(N_CONT_LIST_FEATURES):
        feature_list = []
        for irow in range(N_ROWS):
            n_elements = np.random.choice(list(range(20)))
            feature_list.append(np.random.uniform(-1, 1, size=n_elements).tolist())
        dict_features["cont_list" + str(icontlist)] = feature_list
    for icatlist in range(N_CAT_LIST_FEATURES):
        feature_list = []
        for irow in range(N_ROWS):
            n_elements = np.random.choice(list(range(20)))
            feature_list.append(np.random.choice(list(range(10)), size=n_elements).tolist())
        dict_features["cat_list" + str(icatlist)] = feature_list
    dict_features["label"] = np.random.choice(list(range(2)), size=N_ROWS)
    df = pd.DataFrame(dict_features)
    return df

We can configure the size of the dataset and numbers of features of the different type. As this is just a example, we use only 20,000 rows.

In [5]:
N_ROWS = 20000
N_CONT_FEATURES = 5
N_CAT_FEATURES = 7
N_CONT_LIST_FEATURES = 2
N_CAT_LIST_FEATURES = 3

In [6]:
df = create_synthetic_df(
    N_CONT_FEATURES, N_CAT_FEATURES, N_CONT_LIST_FEATURES, N_CAT_LIST_FEATURES, N_ROWS
)

We can take a look on the dataset.

In [7]:
df.head()

,cont0,cont1,cont2,cont3,cont4,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cont_list0,cont_list1,cat_list0,cat_list1,cat_list2,label
0,-0.397172,0.532571,0.759104,0.594407,-0.524947,8,7,2,1,0,5,0,"[-0.6707850610776214, -0.05450009496947694, -0...","[0.5190079351542354, -0.3838151710281379, 0.44...","[1, 0, 1]",[],"[7, 1, 9, 8, 0, 8, 6, 7, 2, 8]",0
1,0.862493,-0.109889,-0.603772,0.057867,0.467439,0,3,8,8,4,0,9,"[-0.05660807519951194, -0.07055138523986693, 0...","[-0.6486521298228771, -0.8936482314761995, -0....","[7, 1, 3, 8, 8, 9, 8, 9, 0, 5, 8, 4, 6, 4, 8, 0]","[1, 0, 5, 4, 3]","[3, 0, 7, 4, 8, 6, 6]",0
2,-0.044618,-0.087187,0.230690,-0.509112,-0.697787,0,7,0,2,5,5,9,"[-0.8096748535850016, 0.5605113724404849, -0.1...","[-0.6596240142354801, 0.8900409553874395, -0.9...","[9, 1, 2, 5, 3, 2, 1]","[4, 3, 9, 7, 3, 9, 4, 0, 8, 0, 1, 4, 0, 1, 2, ...","[8, 4, 6, 6, 7, 3, 0]",0
3,0.326872,0.843668,0.784894,-0.827104,0.754604,9,1,2,9,8,5,6,"[-0.6567031654231053, 0.7465280775584306, -0.4...","[0.3959883972561651, 0.6893892305272766, 0.037...","[4, 0, 8, 4, 1, 7, 1, 9, 3, 9, 9, 8, 4]","[1, 4, 1, 0, 6, 3, 9, 9, 5, 7, 3, 5, 9]","[9, 5, 4, 1, 8, 0, 9, 6, 0, 6, 4, 7, 6, 5, 8, ...",1
4,0.790391,-0.497946,0.393088,0.511822,0.178066,4,5,1,4,7,6,4,"[-0.7939317698805488, -0.3741631460119641, -0....","[-0.5244749482733557, 0.8530664847999185]","[7, 4, 3, 8, 3]",[5],"[2, 4, 6, 3, 9, 4, 6]",0


In [8]:
CONTINUOUS_COLUMNS = ["cont" + str(i) for i in range(N_CONT_FEATURES)]
CATEGORICAL_COLUMNS = ["cat" + str(i) for i in range(N_CAT_FEATURES)]
CONTINUOUS_LIST_COLUMNS = ["cont_list" + str(i) for i in range(N_CONT_LIST_FEATURES)]
CATEGORICAL_LIST_COLUMNS = ["cat_list" + str(i) for i in range(N_CAT_LIST_FEATURES)]
LABEL_COLUMNS = ["label"]

## Convert the Synthetic Dataset into TFRecords

After we created the synthetic dataset, we store it to tfrecords.

In [9]:
import tensorflow as tf

2021-09-01 23:34:16.262872: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [10]:
import os
import multiprocessing as mp
from itertools import repeat


def transform_tfrecords(
    df,
    PATH,
    CONTINUOUS_COLUMNS,
    CATEGORICAL_COLUMNS,
    CONTINUOUS_LIST_COLUMNS,
    CATEGORICAL_LIST_COLUMNS,
    LABEL_COLUMNS,
):
    write_dir = os.path.dirname(PATH)
    if not os.path.exists(write_dir):
        os.makedirs(write_dir)
    file_idx, example_idx = 0, 0
    writer = get_writer(write_dir, file_idx)
    column_names = [
        CONTINUOUS_COLUMNS,
        CATEGORICAL_COLUMNS + LABEL_COLUMNS,
        CONTINUOUS_LIST_COLUMNS,
        CATEGORICAL_LIST_COLUMNS,
    ]
    with mp.Pool(8, pool_initializer, column_names) as pool:
        data = []
        for col_names in column_names:
            if len(col_names) == 0:
                data.append(repeat(None))
            else:
                data.append(df[col_names].values)
        data = zip(*data)
        record_map = pool.imap(build_and_serialize_example, data, chunksize=200)
        for record in record_map:
            writer.write(record)
            example_idx += 1
    writer.close()


def pool_initializer(num_cols, cat_cols, num_list_cols, cat_list_cols):
    global numeric_columns
    global categorical_columns
    global numeric_list_columns
    global categorical_list_columns
    numeric_columns = num_cols
    categorical_columns = cat_cols
    numeric_list_columns = num_list_cols
    categorical_list_columns = cat_list_cols


def build_and_serialize_example(data):
    numeric_values, categorical_values, numeric_list_values, categorical_list_values = data
    feature = {}
    if numeric_values is not None:
        feature.update(
            {
                col: tf.train.Feature(float_list=tf.train.FloatList(value=[val]))
                for col, val in zip(numeric_columns, numeric_values)
            }
        )
    if categorical_values is not None:
        feature.update(
            {
                col: tf.train.Feature(int64_list=tf.train.Int64List(value=[val]))
                for col, val in zip(categorical_columns, categorical_values)
            }
        )
    if numeric_list_values is not None:
        feature.update(
            {
                col: tf.train.Feature(float_list=tf.train.FloatList(value=val))
                for col, val in zip(numeric_list_columns, numeric_list_values)
            }
        )
    if categorical_list_values is not None:
        feature.update(
            {
                col: tf.train.Feature(int64_list=tf.train.Int64List(value=val))
                for col, val in zip(categorical_list_columns, categorical_list_values)
            }
        )
    return tf.train.Example(features=tf.train.Features(feature=feature)).SerializeToString()


def get_writer(write_dir, file_idx):
    filename = str(file_idx).zfill(5) + ".tfrecords"
    return tf.io.TFRecordWriter(os.path.join(write_dir, filename))

We define the output path.

In [11]:
PATH = "/raid/tfrecord-test/"

In [12]:
!rm -rf $PATH
!mkdir $PATH

In [13]:
transform_tfrecords(
    df,
    PATH,
    CONTINUOUS_COLUMNS,
    CATEGORICAL_COLUMNS,
    CONTINUOUS_LIST_COLUMNS,
    CATEGORICAL_LIST_COLUMNS,
    LABEL_COLUMNS,
)

We can check the file.

In [14]:
!ls $PATH

00000.tfrecords


## Convert TFRecords to parquet files

Now, we have a dataset in the tfrecords format. Let's use the `convert_tfrecords_to_parquet` function to convert a tfrecord file into parquet.

In [15]:
import glob

from nvtabular.framework_utils.tensorflow.tfrecords_to_parquet import convert_tfrecords_to_parquet

Let's select all TFRecords in the folder.

In [16]:
filenames = glob.glob(PATH + "/*.tfrecords")

Let's call the `convert_tfrecords_to_parquet`.<br><br>
Some details about the parameters:
* `compression_type` is the compression type of the tfrecords. Options: `""` (no compression), `"ZLIB"`, or `"GZIP"`
* `chunks` defines how many data points per `parquet` file should be saved. It splits a tfrecords into multiple parquet files.
* `convert_lists` defines, if feature lists should be converted into muliple feature columns. Even single dataframe series are 1 dimensional arrays when converted back from tfrecords to parquet.   

In [17]:
convert_tfrecords_to_parquet(
    filenames=filenames, output_dir=PATH, compression_type="", chunks=1000, convert_lists=True
)

2021-09-01 23:34:39.728120: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-09-01 23:34:39.730165: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:0b:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2021-09-01 23:34:39.730197: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-09-01 23:34:39.730269: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-09-01 23:34:39.730305: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2021-09-01 23:34:39.730341: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcufft.

## Let's take a look

We can see that `convert_tfrecords_to_parquet` created multiple files per `tfrecord` depending on the chunk size.

In [18]:
filenames = glob.glob(PATH + "/*.parquet")
filenames

['/raid/tfrecord-test/00000_1.parquet',
 '/raid/tfrecord-test/00000_9.parquet',
 '/raid/tfrecord-test/00000_14.parquet',
 '/raid/tfrecord-test/00000_2.parquet',
 '/raid/tfrecord-test/00000_17.parquet',
 '/raid/tfrecord-test/00000_6.parquet',
 '/raid/tfrecord-test/00000_0.parquet',
 '/raid/tfrecord-test/00000_19.parquet',
 '/raid/tfrecord-test/00000_3.parquet',
 '/raid/tfrecord-test/00000_15.parquet',
 '/raid/tfrecord-test/00000_7.parquet',
 '/raid/tfrecord-test/00000_4.parquet',
 '/raid/tfrecord-test/00000_11.parquet',
 '/raid/tfrecord-test/00000_8.parquet',
 '/raid/tfrecord-test/00000_12.parquet',
 '/raid/tfrecord-test/00000_18.parquet',
 '/raid/tfrecord-test/00000_5.parquet',
 '/raid/tfrecord-test/00000_16.parquet',
 '/raid/tfrecord-test/00000_13.parquet',
 '/raid/tfrecord-test/00000_10.parquet']

If we load the first file, we cann see, that it has the same structure as our original synthetic dataset.

In [19]:
df = pd.read_parquet(filenames[0])
df.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat_list0,cat_list1,cat_list2,cont0,cont1,cont2,cont3,cont4,cont_list0,cont_list1,label
0,7,1,1,6,6,5,5,"[0, 9, 2]","[7, 9, 0, 1, 1, 3, 5, 5]","[5, 3, 5, 1, 8, 4, 2, 3, 5, 1, 6, 7, 5, 5, 1, ...",0.949754,-0.748696,0.835813,-0.444217,0.907264,"[-0.11095038, -0.26764223, -0.7053523, -0.8485...","[-0.059809078, -0.022873674, -0.7533006, -0.20...",0
1,5,4,2,0,0,6,8,"[8, 6, 5, 3, 6, 1, 9, 8, 1, 8, 5, 8, 7, 0, 5, ...","[3, 8, 0, 0, 8, 1, 6, 0]","[2, 8, 2, 1, 1, 9, 0, 2, 9, 5, 1, 6, 4, 3, 6, 4]",0.483914,-0.067152,0.717157,-0.530880,-0.458823,"[-0.63131815, -0.39890215, 0.2429156, -0.36485...","[-0.15147838, -0.50006855, -0.28108948, -0.313...",0
2,0,2,0,6,3,7,0,"[5, 5, 3, 3, 3, 1, 3, 5, 3, 0, 1, 6, 2, 5, 3, ...","[0, 2, 3, 7, 1, 4, 0, 8, 8, 2, 5, 6]","[7, 3, 9, 4, 0, 2, 4, 1, 7, 3, 3]",-0.623489,-0.409438,0.376929,0.011878,0.246547,"[0.5586857, 0.69241923, -0.47191077, 0.3478763...","[0.20118928, 0.254733, -0.59162587, -0.8194611...",0
3,5,5,4,4,7,0,1,"[4, 2, 6, 6, 1, 4, 7, 7, 1, 8, 2, 4, 8, 8, 0]","[9, 8, 4, 7, 0, 4, 5, 7, 1, 4, 8, 3, 1, 1, 5, ...","[6, 4, 5, 9, 8, 3, 2, 9, 0, 7, 0, 1, 3, 9, 0, ...",0.015761,0.559979,0.580781,-0.555637,0.755852,"[0.102869034, 0.78194165, -0.5718406, 0.529719...","[0.48339945, -0.43163055, -0.04833159, 0.37615...",1
4,6,1,3,4,2,7,8,"[2, 4, 3, 3, 3, 0, 3, 6, 4, 7, 9, 0, 3, 5, 5]",[],"[6, 7, 0, 6, 2, 8, 2, 5, 3, 9, 9, 3, 5]",-0.602396,-0.190180,0.340079,-0.125164,-0.455232,"[-0.6019757, 0.43704405]","[0.3293641, -0.97888887, -0.96398735, -0.61759...",0
